In [240]:
import numpy as np
from sklearn.datasets import make_regression

In [241]:
X, y = make_regression(n_samples=1000, n_features=14, n_informative=10, noise=15, random_state=42)
X = np.array(X)
y = np.array(y)

In [267]:
# Линейная регрессия
class MyLineReg():
    # Инициализация класса
    def __init__(self, weights=None, 
                 n_iter: int=100, 
                 learning_rate: int=0.1, 
                 metric: str=None,
                 reg: str=None,
                 l1_coef: float=0,
                 l2_coef: float=0) -> None:
        self.weights = weights
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.metric = metric
        self.reg = reg
        self.regflag = self.reg in ['l1', 'l2', 'elasticnet']
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef
        self.best_score = 0
        
        
    # Вывод информации при передаче в print()
    def __str__(self) -> str:
        return f"MyLineReg class: n_iter={self.n_iter}, learning_rate={self.learning_rate}"

    # Алгоритм тренировки модели
    def fit(self, X: np.ndarray, y: np.ndarray, verbose: int=0) -> None:
        # Дополняем матрицу фичей столбцом единиц слева для w0
        X = np.concatenate((np.ones(len(X))[:, np.newaxis], X), axis=1)
        self.weights = np.ones(X.shape[1])
        
        # процесс обучения
        for i in range(1, self.n_iter + 1):
            pred = np.matmul(X, self.weights.T)
            # Подсчет ошибки
            MSE = np.sum(np.power(pred - y, 2)) / len(pred)
            # Логи
            if verbose and i % verbose == 0:
                print(f"Iteration {i}, loss: {MSE}, {self.metric}: {self.__calc_metrics(pred, y)}")
            # подсчет градиента и обновление весов
            gradient = (2 / len(pred)) * np.matmul((pred - y).T, X)
            self.weights = self.weights - self.learning_rate * gradient - \
                self.regflag * self.__calc_reg(gradient=True)
        
        pred = np.matmul(X, self.weights.T)
        self.best_score = self.__calc_metrics(pred, y)
    
    # Получение весов модели
    def get_coef(self) -> np.ndarray:
        return self.weights[1:]
    
    # Предсказания
    def predict(self, X: np.ndarray) -> np.ndarray:
        X = np.concatenate((np.ones(len(X))[:, np.newaxis], X), axis=1)
        return np.matmul(X, self.weights.T)
    
    # Метрики
    def __calc_metrics(self, pred: np.ndarray, y: np.ndarray) -> np.float64:
        if self.metric not in ['mae', 'mse', 'rmse', 'mape', 'r2']:
            return None
        
        elif self.metric == 'mae':
            return np.sum(np.abs(pred - y)) / len(pred)
        elif self.metric == 'mse':
            return np.sum(np.power(pred - y, 2)) / len(pred)
        elif self.metric == 'rmse':
            return np.sqrt(np.sum(np.power(pred - y, 2)) / len(pred))
        elif self.metric == 'r2':
            return 1 - (np.sum(np.power(y - pred, 2)) / np.sum(np.power(y - y.mean(), 2)))
        elif self.metric == 'mape':
            return np.sum(np.abs((y - pred) / y)) / len(pred) * 100

    # лучший результат модели
    def get_best_score(self) -> np.float64:
        return self.best_score
    
    # подсчет регуляризации
    def __calc_reg(self, gradient: bool=False) -> np.ndarray:        
        l1 = np.sum(np.abs(self.weights))
        l1dif = np.sign(self.weights) * self.l1_coef
        
        l2 = np.sum(np.power(self.weights, 2)) * self.l2_coef
        l2dif = 2 * self.weights * self.l2_coef
        if not gradient:
            if self.reg == 'l1':
                return l1
            elif self.reg == 'l2':
                return l2
            return l1 + l2
        if self.reg == 'l1':
            return l1dif
        elif self.reg == 'l2':
            return l2dif
        return l1dif + l2dif
            
    

In [270]:
linear = MyLineReg(n_iter=50, learning_rate=0.1, metric='mae', reg='l2', l1_coef=0.01)
linear.fit(X, y, verbose=10)
linear.get_coef()

Iteration 10, loss: 657.737973587785, mae: 20.292259499795573
Iteration 20, loss: 231.5100242125879, mae: 12.215005097465703
Iteration 30, loss: 223.84038255545846, mae: 12.005051555730645
Iteration 40, loss: 223.66948016014277, mae: 11.994186476564652
Iteration 50, loss: 223.66509145066428, mae: 11.992771013940239


array([42.91808755, 16.5964704 ,  0.49791346, 65.37196771, 47.59106343,
       61.77630297,  0.24626517, -0.1078105 , 60.01221651, 53.88959075,
       -0.47290879, 14.42682619, 17.68272317, 47.31528971])